In [25]:
import json
import glob

In [26]:
def flatten(l):
    flat_list = [item for sublist in l for item in sublist]
    return flat_list

In [44]:
def read_partitions():
    partition_paths = glob.glob('/Users/sophierand/RCCustomers/customers/USDA/partitions/*.json')
    partition_paths
    all_partitions = []
    for p in partition_paths:
        with open(p) as json_file:
            partition_data = json.load(json_file)
            all_partitions.append(partition_data)
    all_partitions = flatten(all_partitions)
    return all_partitions

def align_md(a):
    metadata_dict = {k:a[k] for k in ['datasets','title']}
    metadata_dict['title'] = metadata_dict['title'].replace('"',"'")
    try:
        url = a['europepmc']['url']
    except:
        try:
            url = a['original']['url']
        except:
            url = ''
    metadata_dict.update({'url':url})
    try:
        pdf = a['original']['pdf']

    except:
        try:
            pdf = a['dimensions']['linkout']
            metadata_dict.update({'pdf':pdf})
        except:
            pdf = ''  
    metadata_dict.update({'pdf':pdf})
    try:
        journal = a['dimensions']['journal_title']

    except:
        try:
            journal = a['europepmc']['journal']
        except:
            try:
                journal = a['original']['journal']
            except:
                journal = ''
    metadata_dict.update({'journal':journal})
    try:
        keywords = a['dimensions']['keywords']

    except:
        keywords = ''
    metadata_dict.update({'keywords':keywords})
    try:
        doi = a['dimensions']['doi']
       
    except:
        try:
            doi = a['europepmc']['doi']
        except:
            try:
                doi = a['original']['doi']
            except:
                doi = ''
    metadata_dict.update({'doi':doi})
    return metadata_dict


import hashlib

def get_hash (strings, prefix=None, digest_size=10):
    """
    construct a unique identifier from a collection of strings
    """
    m = hashlib.blake2b(digest_size=digest_size)
    
    for elem in sorted(map(lambda x: x.encode("utf-8").lower().strip(), strings)):
        m.update(elem)

    if prefix:
        id = prefix + m.hexdigest()
    else:
        id = m.hexdigest()

    return id




In [45]:
parts = read_partitions()
# len(parts)

In [46]:
# parts[1]

### MAIN

In [47]:
aligned_list = []
for u in parts:
    align_return = align_md(u)
    hash_val = 'publication-{}'.format(get_hash(u['title']))
    align_return.update({'pub_id':hash_val})
    aligned_list.append(align_return)

In [48]:
aligned_list[0]

{'datasets': ['dataset-025'],
 'title': 'The Association Between Nutrition Information Use and the Healthfulness of Food Acquisitions',
 'url': 'https://www.ers.usda.gov/webdocs/publications/88531/err-247.pdf?v=0',
 'pdf': '',
 'journal': 'United States Department of Agriculture, Economic Research Service',
 'keywords': '',
 'doi': 'https://www.ers.usda.gov/publications/pub-details/?pubid=88530',
 'pub_id': 'publication-ac36c0758a70132f71c3'}

In [49]:
partition_path = "/Users/sophierand/RCCustomers/customers/USDA/exports/usda_pubs_metadata.json"
with open(partition_path, 'w') as outfile:
    json.dump(aligned_list, outfile, indent=2)
